# Tutorial for Component Map-Making (Written by Mathias Regnier)

## Frequency point of view

In this notebook, we will try to introduce some aspects of component map-making. In general, CMB experiments solve their inverse problems to reconstruct N frequency maps, then apply a component separation method to produce a final cosmological analysis. Here we will join the map-making and component separation steps.

Our data can be described by : 

$$\vec{d} = H \cdot \vec{s} + \vec{n}$$

where $H$ is the pointing matrix, $\vec{s}$ is the input sky and $\vec{n}$ a noise vector. The pointing matrix caracterising the full instrument and contains all the specificities. 

## Component point of view

Now, we would like to think not about frequency maps, but component maps directly. A sky with mixed components can be written by :

$$\vec{s} = A \cdot \vec{c}$$

where A is the mixing matrix and $\vec{c}$ is the components. By replacing this equation in the first one, we have :

$$\vec{d} = H \cdot A \cdot \vec{c} + \vec{n}$$

## QUBIC Operator

The QUBIC operator was build to simulate the way that the instrument is taking data. You can see the code in the QUBIC soft in the `acquisiton.py` at the line 341. By doing that, you can write :

$$\vec{d} = H^{Q} (\vec{x})$$

where $\vec{d}$ is a TOD, $H^{Q}$ is the QUBIC operator and $\vec{x}$ a map in pixel space with shape (Npix, 3). 

## Python code

In [ ]:
import qubic
import sys
sys.path.append('/Users/mregnier/Desktop/PhD Regnier/mypackages')
import Acquisition as acq
import component_model as c
import numpy as np
from importlib import reload
from pyoperators import *
import matplotlib.pyplot as plt
reload(acq)

def get_dictionary(nsub, nside, pointing, band):
    dictfilename = 'dicts/pipeline_demo.dict'
    
    # Read dictionary chosen
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictfilename)
    d['nf_recon'] = nsub
    d['nf_sub'] = nsub
    d['nside'] = nside
    d['RA_center'] = 0
    d['DEC_center'] = -57
    center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
    d['effective_duration'] = 3
    d['npointings'] = pointing
    d['filter_nu'] = int(band*1e9)
    d['photon_noise'] = False
    d['config'] = 'FI'
    d['MultiBand'] = True
    
    return d

Nsub=2
nside = 256
pointing = 999

d150 = get_dictionary(Nsub, nside, pointing, 150)

In [ ]:
comp = [c.CMB(), c.Dust(nu0=353, temp=20)]

instance = acq.QubicIntegratedComponentsMapMaking(d150, Nsub=Nsub, comp=comp)
H = instance.get_operator(np.array([1.54]), convolution=False, )
print('*********************************************')
print(f'\nShapein = {H.shapein}, Shapeout = {H.shapeout}\n')
print('*********************************************')
H

In [ ]:
components = instance.get_PySM_maps({'cmb':42, 'dust':'d0'}, r=0.01)

d = H(components)

You should see that this H operator is taking as input maps with shape (Ncomp, Npix, 3) with Ncomp your number of components and Npix the number of pixels of your map. The output is your TOD with shape (Ndets*Nsamples). Now the model is defined based on component maps instead of frequency maps. 

If you look inside the operator, you should see few lines with `DenseOperator`. This line define the mixing matrix  which tells you how your components are mixing at a given frequency. Those numbers are calculating thanks to FG-Buster (PUT LINK) packages which provides foreground models.

In the first lines, you should see a line with `DiagonalOperator` with numbers close to 1, those are the gain detector which are set to be very close to 1 at the beginning. During the convergence that we will see later, we will fit those numbers to marginalized over foregrounds and gain detectors.

$$H(\vec{s}) \longrightarrow H(\vec{c}, \vec{\beta}, \vec{g})$$

As expressed above, the new QUBIC operator is not longer depending of frequency maps but it depends on the components, the spectral indices and the detectors gain which will be a variable to marginalized over them.

## Alternate PCG

For using this method, we used `PyOperators` written by Pierre Chanial (PUT LINK). In this package is define an iterative algorithm called Preconditionning Conjugate Gradient (PCG) which solved linear equation $A x = b$ iteratively. 

+ Choose an initial guess $x_0$
+ Compute first residuals $r_0 = b - A x_0$
+ Compute the first direction $d_0 = M^{-1} r_0$
+ For each k-th step :
    + Compute step size $\alpha_i = \frac{r_i^T M^{-1} r_i}{d_i^T A d_i}$
    + Compute new maps $x_{i+1} = x_i + \alpha_i d_i$
    + Compute new residuals $r_{i+1} = r_i - \alpha_i A d_i$
    + Compute the next direction $d_{i+1} = M^{-1} r_{i+1}$
    
That algorithm is working by minimizing the residuals. For each iterations, it will propose a new solution and compute from that the direction in parameter space where the residuals are smaller than the previous step.

If we go back to our problem, the main equation to solve is $A x = b$ where $x$ is the components. By defining $A = H^T \cdot N^{-1} \cdot H$ and $b = H^T \cdot N^{-1} \cdot \vec{d}$, we can solve the inverse problem.

The PCG is considering constant $H$ (so constant $A$ and $b$) during the convergence so we need to make an alternate PCG by apply the previous algorithm on few steps (no less than 10 due to the convergence step size) and update regularly the spectral indices and gain detectors. 

Our fitting scheme becomes :

+ Define initial guess $\vec{c}_0$, $\vec{\beta}_0$ and $\vec{g}_0$

+ for each k-th step :
    + Define operator $H(\vec{c}_k, \vec{\beta}_k, \vec{g}_k)$
    + Compute $A$ and $b$ term
    + Let PCG converge by applying previous algorithm
    + Fit gain detectors $\vec{g}_{k+1}$
    + Fit spectral indices $\vec{\beta}_{k+1}$
    
    + $\vec{c}_{k} \rightarrow \vec{c}_{k+1}$
    + $\vec{g}_{k} \rightarrow \vec{g}_{k+1}$
    + $\vec{\beta}_{k} \rightarrow \vec{\beta}_{k+1}$


## How to fit our parameters ?

### Gain detectors 

$$\vec{d} = \vec{g} \cdot \vec{d}_{\text{intercalibrated}}$$



$$\chi^2 = (\vec{d} - \vec{d}_{intercalibrated})^T \cdot (\vec{d} - \vec{d}_{intercalibrated})$$
$$\chi^2 = (\vec{d} - g * H * c )^T \cdot (\vec{d} - g * H * c )$$
$$\chi^2 = \vec{d}^T \vec{d} - g^T H^T c^T \vec{d} - \vec{d} g H c + g^T H^T c^T c H g$$

We are looking for a solution which minimize the $\chi^2$ on the gain, so we pose $\frac{\partial \chi^2}{\partial g^T} = 0$, so :

$$0 = - H^T c^T \vec{d} + H^T c^T c H g$$
$$H^T c^T \vec{d} = H^T c^T c H g$$
$$ g = \frac{\vec{d}}{H c}$$

The gain of the detector can be find by compute the ratio between the observation data and the simulated data assuming $g_i = 1$

In [ ]:
def give_me_intercal(D, d):
    return 1/np.sum(D[:]**2, axis=1) * np.sum(D[:] * d[:], axis=1)

R = ReshapeOperator((992,999), 991008)

g = np.random.randn(992) * 0.5 + 1

G = DiagonalOperator(g, broadcast='rightward', shapein=R.shapein)

Hp = G * R.T * H

dp = Hp(components)

plt.figure(figsize=(15, 5))
plt.plot(R.T(d)[0])
plt.plot(dp[0], alpha=0.4)
plt.show()

In [ ]:
grecon = give_me_intercal(R.T(d), dp)

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(2, 2, 1)
plt.scatter(g, grecon, s=5)
plt.plot([0, 2], [0, 2], '--k')
plt.xlabel(r'g', fontsize=12)
plt.ylabel(r'$g_{recon}$', fontsize=12)

plt.subplot(2, 2, 2)
plt.hist(g-grecon)

plt.subplot(2, 2, (3, 4))
plt.scatter(np.arange(0, 992, 1), g, color='blue', label='g', s=20)
plt.scatter(np.arange(0, 992, 1), grecon, marker='x', color='red', s=10, label=r'$g_{recon}$')
plt.xlabel(r'# of detector', fontsize=12)
plt.ylabel(r'gain', fontsize=12)
plt.legend()
plt.show()

### Spectral indices

The more important fit we want to perform is the fit of the spectral indices which describe how the components evolving with respect to the frequency. New, the QUBIC operator is depending of those parameters. In other words, we have a way to simulate data parametrized by spectral indices, and on the other side we have data to compare with. We can apply a $\chi^2$ minimizatioon like :

$$\chi^2 = (\vec{d} - H(\beta) \cdot \vec{c}_i)^T \cdot (\vec{d} - H(\beta) \cdot \vec{c}_i)$$

In [ ]:
def myChi2(x):

    H_i = instance.update_A(H, x)

    fakedata = H_i(components)
            
    return np.sum((fakedata/fakedata.max() - d/d.max())**2)


allbeta = np.linspace(1.5, 1.6, 100)
allchi2 = np.zeros(allbeta.shape[0])
for ii, i in enumerate(allbeta):
    allchi2[ii] = myChi2(np.array([i]))

In [ ]:
plt.plot(allbeta, allchi2)
plt.axvline(allbeta[np.where(allchi2 == allchi2.min())[0]])

In [ ]:
prob = np.exp(-allchi2)/allchi2
plt.plot(allbeta, prob/prob.max())
plt.xlim(1.5, 1.6)
plt.xlabel(r'$\beta$', fontsize=14)
plt.ylabel(r'$P(\beta)$', fontsize=14)

In [ ]:
allchi2

In [ ]:
from scipy.optimize import minimize

minimize(myChi2, x0=np.ones(1), method='L-BFGS-B', tol=1e-10)

In [ ]:
import pysm3
import healpy as hp

sky = pysm3.Sky(nside=256, preset_strings=['d1'])
beta = np.array(sky.components[0].mbb_index)

hp.mollview(beta, cmap='jet', min=1.4, max=1.65, title=f'Spectral Index - {beta.shape[0]}')

In [ ]:
hp.mollview(components[0, :, 0])

# Let's see what kind of results we have 

In [ ]:
import healpy as hp
import os.path as op
alm = hp.sphtfunc.map2alm(components[0].T, lmax=2*256)
cl = hp.sphtfunc.alm2cl(alm)[:, 40:2*256]
ell = np.arange(40, 2*256, 1)
ell_true = np.arange(2, 4000, 1)

def cl2dl(ell, cl):

    dl=np.zeros(ell.shape[0])
    for i in range(ell.shape[0]):
        dl[i]=(ell[i]*(ell[i]+1)*cl[i])/(2*np.pi)
    return dl

CMB_CL_FILE = op.join('Cls_Planck2018_%s.fits')

def _get_Cl_cmb(Alens, r):
    power_spectrum = hp.read_cl(CMB_CL_FILE%'lensed_scalar')[:,:4000]
    if Alens != 1.:
        power_spectrum[2] *= Alens
    if r:
        power_spectrum += r * hp.read_cl(CMB_CL_FILE%'unlensed_scalar_and_tensor_r1')[:,:4000]
    return power_spectrum

dl = cl2dl(ell, cl[2])
cl_true = _get_Cl_cmb(Alens=1, r=0.01)[2, :]
dl_true = cl2dl(ell_true, cl_true)

In [ ]:
plt.plot(ell, dl)
plt.plot(ell_true, dl_true)
plt.yscale('log')
plt.ylim(1e-4, 1e-1)
plt.xlim(30, 550)

In [ ]:
from scipy.optimize import minimize
class Spectra:
    
    def __init__(self, lmin, lmax, dl, r=0, Alens=1, icl=2, CMB_CL_FILE=None):
        self.lmin = lmin
        self.lmax = lmax
        self.icl = icl
        self.r = r
        self.dl = dl
        self.CMB_CL_FILE = CMB_CL_FILE
        self.Alens = Alens
        self.ell_theo = np.arange(2, self.lmax, 1)
        self.cl_theo = self._get_Cl_cmb(r=r)[self.icl]
        self.dl_theo = self._cl2dl(self.ell_theo, self.cl_theo)
        self.ell_obs = np.arange(lmin, lmax, dl)
        
        
    def _get_Cl_cmb(self, r):
        power_spectrum = hp.read_cl(self.CMB_CL_FILE%'lensed_scalar')[:, :self.lmax]
        if self.Alens != 1.:
            power_spectrum[2] *= self.Alens
        if r:
            power_spectrum += r * hp.read_cl(self.CMB_CL_FILE%'unlensed_scalar_and_tensor_r1')[:, :self.lmax]
        return power_spectrum
    
    def _cl2dl(self, ell, cl):
        dl=np.zeros(ell.shape[0])
        for i in range(ell.shape[0]):
            dl[i]=(ell[i]*(ell[i]+1)*cl[i])/(2*np.pi)
        return dl
    
    def binning(self, cl):
        
        nbins = len(self.ell_obs)
        cl_binned = np.zeros(nbins)
        for i in range(nbins):
            cl_binned[i] = np.mean(cl[self.ell_obs[i]-int(self.dl/2) : self.ell_obs[i]+int(self.dl/2)])
        return cl_binned
    
    def get_observed_spectra(self, s):
        
        alm = hp.sphtfunc.map2alm(s, lmax=self.lmax)
        cl = hp.sphtfunc.alm2cl(alm)[self.icl, :]

        #cl_binned = self.binning(cl)
        #print(cl_binned)
        dl_binned = self._cl2dl(self.ell_theo, cl)
        
        #print(dl_binned)
        return dl_binned
    
    def chi2(self, r, dobs):
        #print(r)
        cl = self._get_Cl_cmb(r)[self.icl]
        d = self._cl2dl(self.ell_theo, cl)#[np.array(self.ell_obs, dtype=int)]
        #d = self._cl2dl(self.ell_theo, self.cl_theo)[np.array(self.ell_obs, dtype=int)]
        #print(d)
        #print(dobs)
        return np.sum((d - dobs)**2)
        
    def give_rbias(self, cl_obs):
        
        cl_theo_binned = self.dl_theo.copy()#[np.array(self.ell_obs, dtype=int)]
        s = minimize(self.chi2, x0=np.ones(1), args=(cl_obs), method='TNC', tol=1e-10, bounds=[(0, 1)])
        return s.x[0] - self.r
    

In [ ]:
s = Spectra(40, 2*256, 35, r=0.01, Alens=1, icl=2)
s.give_rbias(s.get_observed_spectra(components[0].T))

In [ ]:
plt.plot(s.ell_theo, s.dl_theo)
plt.plot(s.ell_theo, s.get_observed_spectra(components[0].T), '-k')
plt.yscale('log')
plt.xlim(20, 500)

In [ ]:
s.get_observed_spectra(components[0].T)

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

center = qubic.equ2gal(100, -157)
N = 500
vals = np.ones((N, 4))
vals[:, 0] = np.linspace(90/256, 1, N)*0
vals[:, 1] = np.linspace(39/256, 1, N)
vals[:, 2] = np.linspace(41/256, 1, N)*0
newcmp = ListedColormap(vals)

In [ ]:
plt.figure(figsize=(6, 6))

hp.gnomview(components[1, :, 0], cmap=None, min=0, max=50000, rot=center, reso=15, sub=(2, 2, 1), cbar=False, title='',
           notext=True)
hp.gnomview(components[1, :, 0], cmap=None, min=0, max=50000, rot=center, reso=15, sub=(2, 2, 2), cbar=False, title='',
           notext=True)
hp.gnomview(components[1, :, 0], cmap=None, min=0, max=50000, rot=center, reso=15, sub=(2, 2, 3), cbar=False, title='',
           notext=True)
hp.gnomview(components[1, :, 0], cmap=None, min=0, max=50000, rot=center, reso=15, sub=(2, 2, 4), cbar=False, title='',
           notext=True)

all_comp = [r'$CMB$', r'$A_d$']
all_pol = [r'$I$', r'$Q$', r'$U$']

for i in range(len(all_comp)):
    plt.annotate(all_comp[i], xy=(0, 0), xytext=(1/len(all_comp) - 0.06, 1/(i+1) - 0.06), 
                 xycoords='figure fraction', fontsize=12, ha="center", va="center")

    plt.annotate(all_pol[2], xy=(0, 0), xytext=(0.06, 1/(i+1) - 0.06), 
                xycoords='figure fraction', fontsize=12, ha="center", va="center")

plt.show()

In [ ]:
def get_cl(m):
    alm = hp.sphtfunc.map2alm(m, lmax=2*256)
    cl = hp.sphtfunc.alm2cl(alm)[2]
    return cl
import pickle

pkl_file = open('test_cl.pkl', 'rb')
dataset = pickle.load(pkl_file)

In [ ]:
plt.plot(dataset['ell'], dataset['cl'][0])
plt.plot(s.ell_theo, s.dl_theo)
plt.yscale('log')
plt.ylim(5e-6, 2e1)

In [ ]:
cl = get_cl(np.random.randn(components[0].T.shape[0], components[0].T.shape[1])*1.1)
dl = s._cl2dl(np.arange(0, 2*256, 1), cl)
print(dataset['cl'].shape)
print(dl.shape)

In [ ]:
plt.plot(dataset['ell'], dataset['cl'][0])
plt.plot(np.arange(0, 2*256, 1)[12:], dl[12:])
plt.plot(np.arange(0, 2*256, 1)[12:], dl[12:]-dataset['cl'][0])
plt.plot(s.ell_theo, s.dl_theo)
#plt.yscale('log')
plt.ylim(-0.1, 0.3)#5e-6, 2e1)
plt.xlim(0, 300)

In [ ]:
np.random.randn(components[0].T.shape[0], components[0].T.shape[1])